In [0]:
import pandas as pd
from scipy.spatial.distance import hamming
import Levenshtein as lv
import pickle

In [0]:
cd /home/cfriedline/eckertlab/projects/burt/seq

In [0]:
barcodes = pd.read_excel("barcodes.xls")

In [0]:
plates = !ls plate*.xls

In [0]:
def convert_plate(plate_dict):
    d = {}
    for row, inner in plate_dict.items():
        for col, sample in inner.items():
            key = "{}{}".format(row, col)
            d[key] = sample
    return pd.DataFrame(d, index=["sample"]).T
test = convert_plate(p)

In [0]:
def write_barcode(df, out):
     df[['sample', 'well', 'barcode1', 'barcode2']].to_csv(out, sep="\t", index=False)

In [0]:
def write_gbsx_barcode(df, out):
    d = df.copy()
    d['none'] = ""
    d['gbsx_bc'] = d.barcode2.apply(lambda x: x.replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "").upper())
    d['gbsx_enz'] = "EcoRI"
    d[['sample', 'gbsx_bc', 'gbsx_enz']].to_csv(out, sep="\t", index=False, header=False)

In [0]:
for plate in plates:
    p = pd.read_excel(plate, index_col=0).T.to_dict()
    p = convert_plate(p)
    d = p.merge(barcodes, left_index=True, right_on="well")
    lib =plate.split("_")[1] 
    gbsx = "barcode_{}_gbsx.txt".format(lib)
    d = d.sort_values("well")
    write_gbsx_barcode(d, gbsx)

In [0]:
bcs = barcodes.barcode2.apply(lambda x: x.replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "")[:-1].upper())

In [0]:
bc_lens = {}
for b in bcs:
    l = len(b)
    if not l in bc_lens:
        bc_lens[l] = []
    bc_lens[l].append(b)

In [0]:
pickle.dump(bc_lens, open("bc_lens.pkl", "wb"))